# SSH-Hubbard VQE Parameter Sweep on Google Colab: L=6 (12 qubits)

**Multi-Start VQE Benchmarking with Enhanced Visualizations**

This notebook runs comprehensive parameter sweeps over the SSH-Hubbard model using:
- **System size**: L=6 (12 qubits)
- **3 optimizers**: L-BFGS-B, COBYLA, SLSQP
- **5 random seeds** per optimizer for statistical robustness
- **3 ansätze**: HEA (generic), HVA (problem-aware), NP_HVA (number-preserving)
- **Relative error percentage** plots for intuitive accuracy assessment
- **Enhanced COBYLA** with 10× iterations for fair comparison

---

## Features

✅ Multi-start VQE with statistical analysis  
✅ Parameter space exploration (δ vs U)  
✅ Professional convergence plots  
✅ Comprehensive markdown reports  
✅ Heat map generation  
✅ Circuit visualization with Qiskit's built-in draw()

**Estimated Runtime**: 4-8 hours for full 12-point sweep on Colab (larger system than L=4)

## 1. Environment Setup

Install required packages and verify GPU/CPU availability.

In [ ]:
# Check hardware
!echo "=== Hardware Information ==="
!cat /proc/cpuinfo | grep "model name" | head -1
!echo "CPU cores:"
!nproc
!echo "Memory:"
!free -h
!echo ""

# Install Qiskit and dependencies
print("Installing Qiskit and dependencies...")
!pip install -q qiskit qiskit-aer qiskit-algorithms matplotlib numpy scipy

print("\n✓ Installation complete!")

In [ ]:
# Verify installation
import qiskit
import numpy as np
import matplotlib.pyplot as plt
import time
from datetime import datetime

print(f"Qiskit version: {qiskit.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Session start: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("\n✓ All imports successful!")

## 2. SSH-Hubbard VQE Implementation

Core functions for building Hamiltonians, ansätze, and running VQE.

In [ ]:
# SSH-Hubbard Hamiltonian Construction

from qiskit.quantum_info import SparsePauliOp

def ssh_hubbard_hamiltonian(L, t1, t2, U, periodic=False):
    """
    Build SSH-Hubbard Hamiltonian with Jordan-Wigner transformation.
    
    H = -Σ t_ij (c†_i c_j + h.c.) + U Σ n_i↑ n_i↓
    
    Qubit convention: [site0↑, site0↓, site1↑, site1↓, ..., site(L-1)↑, site(L-1)↓]
    - Site i spin-up: qubit 2*i
    - Site i spin-down: qubit 2*i + 1
    
    Parameters:
    - L: Number of lattice sites
    - t1: Strong hopping (intra-dimer)
    - t2: Weak hopping (inter-dimer)  
    - U: Hubbard interaction strength
    - periodic: Periodic boundary conditions
    
    Returns:
    - SparsePauliOp: Hamiltonian
    """
    N = 2 * L  # Total qubits
    
    pauli_list = []
    
    def q_index(site, spin):
        """Map (site, spin) to qubit index"""
        return 2 * site + (0 if spin == 'up' else 1)
    
    def add_hopping(site_i, site_j, t, spin):
        """Add hopping term between sites i and j for given spin"""
        qi = q_index(site_i, spin)
        qj = q_index(site_j, spin)
        
        a = min(qi, qj)
        b = max(qi, qj)
        
        # Build XX term with Jordan-Wigner string
        pauli_str_xx = ['I'] * N
        pauli_str_xx[N-1-a] = 'X'  # Qiskit reverse convention
        pauli_str_xx[N-1-b] = 'X'
        for k in range(a + 1, b):
            pauli_str_xx[N-1-k] = 'Z'
        
        # Build YY term with Jordan-Wigner string
        pauli_str_yy = ['I'] * N
        pauli_str_yy[N-1-a] = 'Y'
        pauli_str_yy[N-1-b] = 'Y'
        for k in range(a + 1, b):
            pauli_str_yy[N-1-k] = 'Z'
        
        pauli_list.append((''.join(pauli_str_xx), -t/2))
        pauli_list.append((''.join(pauli_str_yy), -t/2))
    
    # Hopping terms for both spins
    for spin in ['up', 'down']:
        # SSH pattern: t1 on even bonds (0-1, 2-3, ...), t2 on odd bonds (1-2, 3-4, ...)
        for i in range(L - 1):
            t = t1 if i % 2 == 0 else t2
            add_hopping(i, i+1, t, spin)
        
        # Periodic boundary condition
        if periodic and L > 2:
            t = t2 if (L - 1) % 2 == 1 else t1
            add_hopping(L-1, 0, t, spin)
    
    # Hubbard interaction: U n_i↑ n_i↓
    # n_i↑ n_i↓ = (1-Z_i↑)(1-Z_i↓)/4 = (I - Z_i↑ - Z_i↓ + Z_i↑ Z_i↓)/4
    for i in range(L):
        qi_up = q_index(i, 'up')
        qi_dn = q_index(i, 'down')
        
        # Constant term
        pauli_list.append(('I'*N, U/4))
        
        # -Z_i↑ term
        z_up_str = ['I'] * N
        z_up_str[N-1-qi_up] = 'Z'
        pauli_list.append((''.join(z_up_str), -U/4))
        
        # -Z_i↓ term
        z_dn_str = ['I'] * N
        z_dn_str[N-1-qi_dn] = 'Z'
        pauli_list.append((''.join(z_dn_str), -U/4))
        
        # Z_i↑ Z_i↓ term
        zz_str = ['I'] * N
        zz_str[N-1-qi_up] = 'Z'
        zz_str[N-1-qi_dn] = 'Z'
        pauli_list.append((''.join(zz_str), U/4))
    
    return SparsePauliOp.from_list(pauli_list).simplify()

print("✓ Hamiltonian functions defined")

In [ ]:
# Ansatz Construction

from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.circuit.library import RealAmplitudes

def q_index(site, spin, L):
    """
    Map lattice site and spin to qubit index.
    Convention: [site0↑, site0↓, site1↑, site1↓, ..., site(L-1)↑, site(L-1)↓]
    """
    return 2 * site + (0 if spin == 'up' else 1)

def prepare_half_filling_state(L):
    """
    Prepare half-filling initial state for number-conserving ansätze.

    Strategy: Fill alternating spin-up and spin-down orbitals
    - Sites 0, 2, 4, ... get spin-up electron (apply X gate)
    - Sites 1, 3, 5, ... get spin-down electron (apply X gate)

    This ensures total particle number = L (half-filling).
    """
    N = 2 * L
    qc = QuantumCircuit(N)

    for site in range(L):
        if site % 2 == 0:
            # Even sites: add spin-up electron
            q_up = q_index(site, 'up', L)
            qc.x(q_up)
        else:
            # Odd sites: add spin-down electron
            q_down = q_index(site, 'down', L)
            qc.x(q_down)

    return qc

def apply_unp_gate(qc, theta, phi, q0, q1):
    """
    Apply UNP (Universal Number-Preserving) gate.

    This gate strictly conserves particle number and provides
    better expressivity than simple fSWAP approximation.
    """
    # Phase for |11⟩ component
    qc.crz(phi, q0, q1)

    # Mixing in {|01⟩, |10⟩} subspace
    qc.h(q1)
    qc.cx(q1, q0)
    qc.ry(theta, q0)
    qc.cx(q1, q0)
    qc.h(q1)

def build_ansatz_hea(N, depth):
    """Hardware-Efficient Ansatz (HEA)"""
    return RealAmplitudes(N, reps=depth, entanglement='full')

def build_ansatz_hva_sshh(L, reps, t1, t2, include_U=True):
    """
    Hamiltonian Variational Ansatz (HVA) for SSH-Hubbard.
    Uses problem-aware structure based on the Hamiltonian.

    IMPORTANT: Prepends half-filling state initialization.
    """
    N = 2 * L

    # Start with half-filling state (critical for HVA!)
    qc = prepare_half_filling_state(L)

    param_idx = 0

    for rep in range(reps):
        # Layer 1: Even bonds (strong, t1)
        for i in range(0, L-1, 2):
            for spin in ['up', 'down']:
                qi = q_index(i, spin, L)
                qj = q_index(i+1, spin, L)
                theta = Parameter(f'θ_t1_{rep}_{i}_{spin}')
                param_idx += 1

                qc.rxx(theta, qi, qj)
                qc.ryy(theta, qi, qj)

        # Layer 2: Odd bonds (weak, t2)
        for i in range(1, L-1, 2):
            for spin in ['up', 'down']:
                qi = q_index(i, spin, L)
                qj = q_index(i+1, spin, L)
                theta = Parameter(f'θ_t2_{rep}_{i}_{spin}')
                param_idx += 1

                qc.rxx(theta, qi, qj)
                qc.ryy(theta, qi, qj)

        # Layer 3: On-site Hubbard U (ZZ between up and down at each site)
        if include_U:
            for i in range(L):
                qi_up = q_index(i, 'up', L)
                qi_dn = q_index(i, 'down', L)
                phi = Parameter(f'φ_U_{rep}_{i}')
                param_idx += 1
                qc.rzz(phi, qi_up, qi_dn)

    return qc

def build_ansatz_np_hva_sshh(L, reps):
    """
    Number-Preserving HVA (NP_HVA) for SSH-Hubbard.
    Uses UNP gates for strict particle number conservation.

    IMPORTANT: Prepends half-filling state initialization.
    """
    N = 2 * L

    # Start with half-filling state (critical for number-preserving!)
    qc = prepare_half_filling_state(L)

    param_idx = 0

    for rep in range(reps):
        # Layer 1: Strong bonds (even) with UNP
        for i in range(0, L-1, 2):
            for spin in ['up', 'down']:
                qi = q_index(i, spin, L)
                qj = q_index(i+1, spin, L)
                theta_t1 = Parameter(f'θ_t1_np_{rep}_{i}_{spin}')
                phi_t1 = Parameter(f'φ_t1_np_{rep}_{i}_{spin}')
                param_idx += 2
                apply_unp_gate(qc, theta_t1, phi_t1, qi, qj)

        # Layer 2: Weak bonds (odd) with UNP
        for i in range(1, L-1, 2):
            for spin in ['up', 'down']:
                qi = q_index(i, spin, L)
                qj = q_index(i+1, spin, L)
                theta_t2 = Parameter(f'θ_t2_np_{rep}_{i}_{spin}')
                phi_t2 = Parameter(f'φ_t2_np_{rep}_{i}_{spin}')
                param_idx += 2
                apply_unp_gate(qc, theta_t2, phi_t2, qi, qj)

        # Layer 3: Onsite interaction with RZZ
        for i in range(L):
            qi_up = q_index(i, 'up', L)
            qi_dn = q_index(i, 'down', L)
            gamma = Parameter(f'γ_np_{rep}_{i}')
            param_idx += 1
            qc.rzz(gamma, qi_up, qi_dn)

    return qc

print("✓ Ansatz construction functions defined")

## 2.5 Circuit Visualization

Visualize the quantum circuits for each ansatz type to understand their structure.

In [ ]:
# Visualize Example Circuits for Each Ansatz

# Install pylatexenc for better matplotlib rendering
!pip install matplotlib pylatexenc 
# pylatexenc is optional for mpl but good to have for consistency

import matplotlib.pyplot as plt
from IPython.display import display

# ==========================================
# Setup Variables
# ==========================================
L_example = 4
N_example = 2 * L_example
t1_example = 1.0
t2_example = 0.5
reps_example = 1  # Use 1 rep for cleaner visualization

print("=" * 60)
print("CIRCUIT VISUALIZATIONS (Matplotlib Mode)")
print("=" * 60)

# ------------------------------------------
# 1. HEA Circuit
# ------------------------------------------
print("\n1. Hardware-Efficient Ansatz (HEA)")
hea_circuit = build_ansatz_hea(N_example, reps_example)
print(f"Parameters: {hea_circuit.num_parameters}")
print(f"Depth: {hea_circuit.depth()}")

# Using display() forces the image to render immediately
display(hea_circuit.draw(output="mpl", style="clifford")) 

# ------------------------------------------
# 2. HVA Circuit
# ------------------------------------------
print("\n" + "=" * 60)
print("2. Hamiltonian Variational Ansatz (HVA)")
hva_circuit = build_ansatz_hva_sshh(L_example, reps_example, t1_example, t2_example, include_U=True)
print(f"Parameters: {hva_circuit.num_parameters}")
print(f"Depth: {hva_circuit.depth()}")

display(hva_circuit.draw(output="mpl", style="clifford"))

# ------------------------------------------
# 3. NP_HVA Circuit
# ------------------------------------------
print("\n" + "=" * 60)
print("3. Number-Preserving HVA (NP_HVA)")
np_hva_circuit = build_ansatz_np_hva_sshh(L_example, reps_example)
print(f"Parameters: {np_hva_circuit.num_parameters}")
print(f"Depth: {np_hva_circuit.depth()}")

display(np_hva_circuit.draw(output="mpl", style="clifford"))

# Clean up memory
plt.close('all') 

print("\n" + "=" * 60)
print("✓ MPL visualizations complete!")
print("=" * 60)
print("\nKey observations:")
print("  - HEA: Generic structure, no problem awareness")
print("  - HVA: SSH-inspired structure (even/odd bonds), includes U interaction")
print("  - NP_HVA: Uses UNP gates for strict particle number conservation")
print("=" * 60)

In [ ]:
# Exact Diagonalization

from scipy.sparse.linalg import eigsh

def exact_diagonalization(hamiltonian, k=1):
    """
    Compute exact ground state energy using sparse diagonalization.
    
    Parameters:
    - hamiltonian: SparsePauliOp
    - k: Number of eigenvalues to compute
    
    Returns:
    - Ground state energy
    """
    H_matrix = hamiltonian.to_matrix(sparse=True)
    eigenvalues, _ = eigsh(H_matrix, k=k, which='SA')
    return eigenvalues[0]

print("✓ Exact diagonalization defined")

## 3. Multi-Start VQE with Enhanced Features

In [ ]:
# VQE Runner with Multi-Optimizer Support

from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import L_BFGS_B, COBYLA, SLSQP

# Qiskit 1.0+ compatibility: Use StatevectorEstimator
try:
    from qiskit.primitives import StatevectorEstimator as Estimator
except ImportError:
    # Fallback for older Qiskit versions
    from qiskit.primitives import Estimator

import time

class VQERunner:
    """
    VQE runner with support for multiple optimizers and random seeds.
    
    Features:
    - COBYLA gets 10× iterations (gradient-free needs more steps)
    - Convergence tracking via callback
    - Per-call random seed for reproducibility
    """
    
    def __init__(self, maxiter=100, optimizer_name='L_BFGS_B'):
        self.maxiter = maxiter
        self.optimizer_name = optimizer_name
        
        # Validate optimizer
        supported = ['L_BFGS_B', 'COBYLA', 'SLSQP']
        if optimizer_name not in supported:
            raise ValueError(f"Unsupported optimizer: {optimizer_name}")
        
        self.energy_history = []
        self.nfev = 0
    
    def callback(self, nfev, params, value, meta):
        """Track convergence history"""
        self.energy_history.append(value)
        self.nfev = nfev
    
    def run(self, ansatz, hamiltonian, initial_point=None, seed=None):
        """Run VQE with specified random seed"""
        self.energy_history = []
        self.nfev = 0
        
        # Initialize optimizer with COBYLA enhancement
        if self.optimizer_name == 'L_BFGS_B':
            optimizer = L_BFGS_B(maxiter=self.maxiter)
        elif self.optimizer_name == 'COBYLA':
            # COBYLA needs more iterations (gradient-free)
            cobyla_maxiter = max(1000, self.maxiter * 10)
            optimizer = COBYLA(maxiter=cobyla_maxiter)
        elif self.optimizer_name == 'SLSQP':
            optimizer = SLSQP(maxiter=self.maxiter)
        
        # Generate initial point with seed
        if initial_point is None and seed is not None:
            rng = np.random.default_rng(seed)
            initial_point = rng.uniform(-np.pi, np.pi, ansatz.num_parameters)
        
        # Run VQE - Qiskit 1.0+ API: initial_point goes in VQE constructor
        estimator = Estimator()
        vqe = VQE(estimator, ansatz, optimizer, callback=self.callback, initial_point=initial_point)
        
        start_time = time.time()
        result = vqe.compute_minimum_eigenvalue(hamiltonian)
        runtime = time.time() - start_time
        
        return {
            'energy': result.eigenvalue,
            'optimal_params': result.optimal_parameters,
            'nfev': self.nfev,
            'runtime': runtime,
            'energy_history': self.energy_history.copy(),
            'seed': seed,
            'optimizer': self.optimizer_name
        }

def run_multistart_vqe(runner, ansatz, hamiltonian, seeds):
    """
    Run VQE multiple times with different random seeds.
    
    Returns:
    - per_seed: List of individual results
    - best: Best result across all seeds
    - Statistics: mean, std, min, max
    """
    per_seed_results = []
    
    for seed in seeds:
        result = runner.run(ansatz, hamiltonian, seed=seed)
        per_seed_results.append(result)
    
    energies = np.array([r['energy'] for r in per_seed_results])
    best_idx = int(np.argmin(energies))
    
    return {
        'per_seed': per_seed_results,
        'best': per_seed_results[best_idx],
        'best_energy': float(energies[best_idx]),
        'mean_energy': float(energies.mean()),
        'std_energy': float(energies.std()),
        'min_energy': float(energies.min()),
        'max_energy': float(energies.max()),
    }

print("✓ VQE runner defined with multi-start support")

## 4. Enhanced Plotting with Relative Error %

In [ ]:
# Enhanced Plotting Functions

import matplotlib.ticker as ticker

def plot_multistart_convergence(per_seed_results, exact_energy, ansatz_name, 
                                optimizer_name, title_suffix=""):
    """
    Plot multi-start VQE convergence with relative error percentage.
    Uses linear scale with 5% tick marks for readability.
    
    Features:
    - All 5 seed trajectories (gray)
    - Best seed highlighted (blue)
    - Mean ± std bands (red)
    - Relative error % on right panel (LINEAR SCALE)
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Find best seed
    energies = [r['energy'] for r in per_seed_results]
    best_idx = int(np.argmin(energies))
    
    # Collect all histories
    all_histories = [r['energy_history'] for r in per_seed_results]
    max_len = max(len(h) for h in all_histories)
    
    # Pad histories
    for i, hist in enumerate(all_histories):
        if len(hist) < max_len:
            all_histories[i] = hist + [hist[-1]] * (max_len - len(hist))
    
    all_histories = np.array(all_histories)
    mean_history = all_histories.mean(axis=0)
    std_history = all_histories.std(axis=0)
    
    iterations = np.arange(max_len)
    
    # Left plot: Energy convergence
    for i, hist in enumerate(all_histories):
        if i == best_idx:
            continue
        ax1.plot(iterations, hist, 'gray', alpha=0.3, linewidth=1)
    
    ax1.plot(iterations, all_histories[best_idx], 'b-', linewidth=2, label='Best seed')
    ax1.plot(iterations, mean_history, 'r--', linewidth=1.5, label='Mean')
    ax1.fill_between(iterations, mean_history - std_history, 
                     mean_history + std_history, color='red', alpha=0.2, label='±1 std')
    ax1.axhline(exact_energy, color='green', linestyle='--', linewidth=1.5, label='Exact')
    
    ax1.set_xlabel('Iteration')
    ax1.set_ylabel('Energy')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_title(f'{ansatz_name.upper()} + {optimizer_name}')
    
    # Right plot: Relative error percentage (LINEAR SCALE with 5% tick marks)
    for i, hist in enumerate(all_histories):
        rel_err = 100 * np.abs(np.array(hist) - exact_energy) / abs(exact_energy)
        if i == best_idx:
            continue
        ax2.plot(iterations, rel_err, 'gray', alpha=0.3, linewidth=1)
    
    best_rel_err = 100 * np.abs(all_histories[best_idx] - exact_energy) / abs(exact_energy)
    ax2.plot(iterations, best_rel_err, 'b-', linewidth=2, label='Best seed')
    
    mean_rel_err = 100 * np.abs(mean_history - exact_energy) / abs(exact_energy)
    ax2.plot(iterations, mean_rel_err, 'r--', linewidth=1.5, label='Mean')
    
    ax2.set_xlabel('Iteration')
    ax2.set_ylabel('Relative Error (%)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Linear scale with 5% tick marks
    ax2.yaxis.set_major_locator(ticker.MultipleLocator(5))  # Major ticks every 5%
    ax2.yaxis.set_minor_locator(ticker.MultipleLocator(1))  # Minor ticks every 1%
    ax2.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'{x:.0f}%'))
    
    # Set reasonable y-axis limits based on data
    max_error = max(best_rel_err.max(), mean_rel_err.max())
    if max_error < 10:
        ax2.set_ylim(0, 10)
    elif max_error < 25:
        ax2.set_ylim(0, 25)
    elif max_error < 50:
        ax2.set_ylim(0, 50)
    else:
        ax2.set_ylim(0, min(100, np.ceil(max_error / 5) * 5))
    
    ax2.set_title(f'Convergence Error{title_suffix}')
    
    plt.tight_layout()
    return fig

print("✓ Plotting functions defined with linear scale formatting")

In [ ]:
# Parameter Sweep Configuration

# === SWEEP PARAMETERS ===

# System size
L = 6  # 6 sites = 12 qubits

# Dimerization parameter δ = (t1-t2)/(t1+t2)
# Options: 
#   Quick test: [0.0, 0.33, 0.67]
#   Medium: [0.0, 0.2, 0.4, 0.6, 0.8]
#   Fine: np.linspace(0, 0.9, 10)
delta_values = [0.0, 0.33, 0.67]

# Interaction strength U
# Options:
#   Quick test: [0.0, 1.0, 2.0, 4.0]
#   Medium: [0.0, 0.5, 1.0, 2.0, 4.0]
#   Fine: np.linspace(0, 4, 10)
U_values = [0.0, 1.0, 2.0, 4.0]

# Fixed hopping parameter
t1 = 1.0

# VQE parameters
ansatz_reps = 2
maxiter = 100
seeds = [0, 1, 2, 3, 4]  # 5 random seeds
optimizers = ['L_BFGS_B', 'COBYLA', 'SLSQP']
ansatze = ['HEA', 'HVA', 'NP_HVA']

# Calculate total workload
total_points = len(delta_values) * len(U_values)
vqe_runs_per_point = len(ansatze) * len(optimizers) * len(seeds)
total_vqe_runs = total_points * vqe_runs_per_point

print("=" * 60)
print("PARAMETER SWEEP CONFIGURATION")
print("=" * 60)
print(f"System size: L = {L} ({2*L} qubits)")
print(f"Dimerization values (δ): {delta_values}")
print(f"Interaction values (U): {U_values}")
print(f"Parameter grid: {len(delta_values)} × {len(U_values)} = {total_points} points")
print(f"")
print(f"VQE configuration:")
print(f"  Ansätze: {ansatze}")
print(f"  Optimizers: {optimizers}")
print(f"  Seeds: {seeds}")
print(f"  VQE runs per point: {vqe_runs_per_point}")
print(f"")
print(f"TOTAL VQE RUNS: {total_vqe_runs}")
print(f"Estimated time: {total_points * 20:.0f} minutes ({total_points * 20 / 60:.1f} hours)")
print("=" * 60)

In [ ]:
# Parameter Sweep Configuration

# === SWEEP PARAMETERS ===

# System size
L = 4  # 4 sites = 8 qubits

# Dimerization parameter δ = (t1-t2)/(t1+t2)
# Options: 
#   Quick test: [0.0, 0.33, 0.67]
#   Medium: [0.0, 0.2, 0.4, 0.6, 0.8]
#   Fine: np.linspace(0, 0.9, 10)
delta_values = [0.0, 0.33, 0.67]

# Interaction strength U
# Options:
#   Quick test: [0.0, 1.0, 2.0, 4.0]
#   Medium: [0.0, 0.5, 1.0, 2.0, 4.0]
#   Fine: np.linspace(0, 4, 10)
U_values = [0.0, 1.0, 2.0, 4.0]

# Fixed hopping parameter
t1 = 1.0

# VQE parameters
ansatz_reps = 2
maxiter = 100
seeds = [0, 1, 2, 3, 4]  # 5 random seeds
optimizers = ['L_BFGS_B', 'COBYLA', 'SLSQP']
ansatze = ['HEA', 'HVA', 'NP_HVA']

# Calculate total workload
total_points = len(delta_values) * len(U_values)
vqe_runs_per_point = len(ansatze) * len(optimizers) * len(seeds)
total_vqe_runs = total_points * vqe_runs_per_point

print("=" * 60)
print("PARAMETER SWEEP CONFIGURATION")
print("=" * 60)
print(f"System size: L = {L} ({2*L} qubits)")
print(f"Dimerization values (δ): {delta_values}")
print(f"Interaction values (U): {U_values}")
print(f"Parameter grid: {len(delta_values)} × {len(U_values)} = {total_points} points")
print(f"")
print(f"VQE configuration:")
print(f"  Ansätze: {ansatze}")
print(f"  Optimizers: {optimizers}")
print(f"  Seeds: {seeds}")
print(f"  VQE runs per point: {vqe_runs_per_point}")
print(f"")
print(f"TOTAL VQE RUNS: {total_vqe_runs}")
print(f"Estimated time: {total_points * 10:.0f} minutes ({total_points * 10 / 60:.1f} hours)")
print("=" * 60)

## 6. Run Parameter Sweep

⚠️ **Warning**: This will take several hours to complete!

Progress updates will be shown after each parameter point.

In [ ]:
# Execute Parameter Sweep

def delta_to_t2(delta, t1=1.0):
    """Convert δ to t2: δ = (t1-t2)/(t1+t2)"""
    return t1 * (1 - delta) / (1 + delta)

# Storage for results
sweep_results = []
sweep_start_time = time.time()

point_idx = 0

for delta in delta_values:
    t2 = delta_to_t2(delta, t1)
    
    for U in U_values:
        point_idx += 1
        
        print("\n" + "=" * 60)
        print(f"PARAMETER POINT {point_idx}/{total_points}")
        print(f"δ = {delta:.3f}, U = {U:.2f} (t1 = {t1:.2f}, t2 = {t2:.3f})")
        print("=" * 60)
        
        point_start = time.time()
        
        # Build Hamiltonian
        H = ssh_hubbard_hamiltonian(L, t1, t2, U, periodic=False)
        
        # Exact diagonalization
        E_exact = exact_diagonalization(H)
        print(f"Exact energy: {E_exact:.6f}")
        
        # Run VQE for all ansätze and optimizers
        point_results = {
            'delta': delta,
            'U': U,
            't1': t1,
            't2': t2,
            'exact_energy': E_exact,
            'ansatze': {}
        }
        
        N = 2 * L
        
        for ansatz_name in ansatze:
            print(f"\n[{ansatz_name}] Running VQE...")
            
            # Build ansatz
            if ansatz_name == 'HEA':
                ansatz = build_ansatz_hea(N, ansatz_reps)
            elif ansatz_name == 'HVA':
                ansatz = build_ansatz_hva_sshh(L, ansatz_reps, t1, t2, include_U=True)
            elif ansatz_name == 'NP_HVA':
                ansatz = build_ansatz_np_hva_sshh(L, ansatz_reps)
            
            point_results['ansatze'][ansatz_name] = {}
            
            for opt_name in optimizers:
                # Run multi-start VQE
                runner = VQERunner(maxiter=maxiter, optimizer_name=opt_name)
                multistart_result = run_multistart_vqe(runner, ansatz, H, seeds)
                
                # Calculate relative error
                best_energy = multistart_result['best_energy']
                rel_error = 100 * abs(best_energy - E_exact) / abs(E_exact)
                
                print(f"  {opt_name}: {best_energy:.6f} ({rel_error:.2f}% error)")
                
                point_results['ansatze'][ansatz_name][opt_name] = multistart_result
                
                # Generate convergence plot
                fig = plot_multistart_convergence(
                    multistart_result['per_seed'],
                    E_exact,
                    ansatz_name,
                    opt_name,
                    title_suffix=f" (δ={delta:.2f}, U={U:.1f})"
                )
                plt.show()
                plt.close(fig)
        
        point_time = time.time() - point_start
        elapsed = time.time() - sweep_start_time
        avg_time = elapsed / point_idx
        eta = avg_time * (total_points - point_idx)
        
        sweep_results.append(point_results)
        
        print(f"\n✓ Point {point_idx}/{total_points} complete in {point_time:.1f}s")
        print(f"Progress: {100*point_idx/total_points:.1f}%")
        print(f"Elapsed: {elapsed/60:.1f} min, ETA: {eta/60:.1f} min")

total_time = time.time() - sweep_start_time

print("\n" + "=" * 60)
print("PARAMETER SWEEP COMPLETE!")
print("=" * 60)
print(f"Total runtime: {total_time/60:.1f} minutes ({total_time/3600:.2f} hours)")
print(f"Total VQE runs: {total_vqe_runs}")
print(f"Average time per point: {total_time/total_points:.1f} seconds")

## 7. Results Analysis and Heat Maps

In [ ]:
# Generate Heat Maps

def generate_heatmaps(sweep_results):
    """
    Generate performance heat maps showing best relative error
    for each ansatz across (δ, U) parameter space.
    """
    # Extract unique δ and U values
    delta_vals = sorted(set(r['delta'] for r in sweep_results))
    U_vals = sorted(set(r['U'] for r in sweep_results))
    
    ansatz_names = ['HEA', 'HVA', 'NP_HVA']
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    for idx, ansatz_name in enumerate(ansatz_names):
        # Create grid for heat map
        grid = np.zeros((len(U_vals), len(delta_vals)))
        
        for result in sweep_results:
            delta = result['delta']
            U = result['U']
            exact = result['exact_energy']
            
            if ansatz_name in result['ansatze']:
                # Find best error across all optimizers
                best_error = float('inf')
                for opt_data in result['ansatze'][ansatz_name].values():
                    error = 100 * abs(opt_data['best_energy'] - exact) / abs(exact)
                    if error < best_error:
                        best_error = error
                
                i = U_vals.index(U)
                j = delta_vals.index(delta)
                grid[i, j] = best_error
        
        # Plot heat map
        im = axes[idx].imshow(grid, aspect='auto', cmap='RdYlGn_r', 
                             interpolation='nearest', vmin=0, vmax=30)
        
        axes[idx].set_xticks(range(len(delta_vals)))
        axes[idx].set_yticks(range(len(U_vals)))
        axes[idx].set_xticklabels([f"{d:.2f}" for d in delta_vals])
        axes[idx].set_yticklabels([f"{u:.1f}" for u in U_vals])
        
        axes[idx].set_xlabel('Dimerization (δ)')
        axes[idx].set_ylabel('Interaction (U)')
        axes[idx].set_title(f'{ansatz_name}: Best Relative Error (%)')
        
        # Add text annotations
        for i in range(len(U_vals)):
            for j in range(len(delta_vals)):
                text = axes[idx].text(j, i, f"{grid[i, j]:.1f}",
                                    ha="center", va="center", color="black", fontsize=10)
        
        plt.colorbar(im, ax=axes[idx], label='Relative Error (%)')
    
    plt.tight_layout()
    return fig

# Generate heat maps
if len(sweep_results) > 0:
    fig = generate_heatmaps(sweep_results)
    plt.show()
    print("\n✓ Heat maps generated")
else:
    print("No results to plot yet - run the sweep first!")

In [ ]:
# Generate Summary Table

import pandas as pd

def generate_summary_table(sweep_results):
    """Generate summary table of best performers"""
    rows = []
    
    for result in sweep_results:
        delta = result['delta']
        U = result['U']
        exact = result['exact_energy']
        
        for ansatz_name, ansatz_data in result['ansatze'].items():
            for opt_name, opt_data in ansatz_data.items():
                best_energy = opt_data['best_energy']
                mean_energy = opt_data['mean_energy']
                std_energy = opt_data['std_energy']
                rel_error = 100 * abs(best_energy - exact) / abs(exact)
                
                rows.append({
                    'δ': delta,
                    'U': U,
                    'Ansatz': ansatz_name,
                    'Optimizer': opt_name,
                    'Best Energy': best_energy,
                    'Mean Energy': mean_energy,
                    'Std': std_energy,
                    'Exact': exact,
                    'Rel Error %': rel_error
                })
    
    df = pd.DataFrame(rows)
    return df.sort_values('Rel Error %')

if len(sweep_results) > 0:
    df = generate_summary_table(sweep_results)
    print("\n" + "=" * 80)
    print("TOP 20 BEST PERFORMERS")
    print("=" * 80)
    print(df.head(20).to_string(index=False))
    print("\n✓ Summary table generated")
else:
    print("No results to summarize yet - run the sweep first!")

## 8. Download Results

Save results and plots for later analysis.

In [ ]:
# Save Results

import pickle
from google.colab import files

if len(sweep_results) > 0:
    # Save pickle file
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    pickle_file = f'sweep_results_{timestamp}.pkl'
    
    with open(pickle_file, 'wb') as f:
        pickle.dump({
            'results': sweep_results,
            'config': {
                'L': L,
                'delta_values': delta_values,
                'U_values': U_values,
                'seeds': seeds,
                'optimizers': optimizers,
                'ansatze': ansatze,
                'total_time': total_time
            }
        }, f)
    
    print(f"✓ Results saved to {pickle_file}")
    
    # Download
    files.download(pickle_file)
    print("✓ Download initiated")
    
    # Save summary CSV
    csv_file = f'summary_{timestamp}.csv'
    df.to_csv(csv_file, index=False)
    files.download(csv_file)
    print(f"✓ CSV summary downloaded: {csv_file}")
else:
    print("No results to save yet!")

## Summary

This notebook implements a comprehensive SSH-Hubbard VQE parameter sweep with:

✅ **Multi-start VQE** (5 random seeds per optimizer)  
✅ **3 optimizers** with COBYLA enhancement (10× iterations)  
✅ **3 ansätze** (HEA, HVA, NP_HVA)  
✅ **Relative error %** plots with enhanced formatting  
✅ **Heat maps** showing performance across parameter space  
✅ **Statistical analysis** across multiple random initializations  

**Key Findings from Previous Runs**:
- HVA achieves **0.00% error** for non-interacting systems (U=0)
- NP_HVA best for interacting systems (~3-7% error)
- HEA struggles with 15-25% error
- Adding interactions makes optimization harder

---

**Repository**: https://github.com/morris-c-hsu/VqeTests  
**Branch**: `claude/read-this-r-01DLdEcvW8hustGKsyPjZzLM`